# Development bidsification

To be run on **test dataset** to configure and test bidsmap and plugins.

 - [Dataset path configuration](./configuration.ipynb): to define working paths
 - [Production version](./bidsification_prod.ipynb): to use when bidsification need to be adjusted
 - [Instalation notebook](installation.ipynb): if bidsme and other packages must be installed/updated

# Initialisation

To run once, after launching notebook or restarting kernel to initialize bidsme and setup paths

> Verify the version of `bidsme` and import errors.

In [ ]:
import shutil
import bidsme

bidsme.init()

Insure that you are working on **Test** dataset (`production=False`).

# Preparation

To run on test dataset when reconfiguration of preparation step is needed

 - [Preparation plugin file](../plugin/prepare_plugin.py)

In [ ]:
# Online help for preparation
?bidsme.prepare

In [ ]:
# Cleanup prepared directory if needed
for f in os.listdir(prepared):
    f = os.path.join(prepared, f)
    if os.path.isdir(f):
        shutil.rmtree(f)
    else:
        os.remove(f)

In [ ]:
# To manually set list of subjects to run with bidisified name, for ex.
# FCAVC012 becomes sub-AVC012
# If list is empty, preparation will run on all aviable subjects
participants = ["sub-SCD001"]

In [ ]:
bidsme.prepare(source, prepared,
               data_dirs={"MRI/s*/nii": "MRI", "VIZ/e*/": "PET", "UCB/e*/": "PET"},
               ses_no_dir=True,
               plugin_file=prepare_plugin,
               part_template=part_template,
               sub_skip_tsv=True, sub_list=participants,
               dry_run=False)

### Manual data curating

 - [MRI_white_list](../lists/MRI_white_list.json) -- list os standard acquisition in session
 - [black_list](../lists/black_list.json) -- list of subjects/sessions to discard
 - [MRI_to_remove](../lists/MRI_to_remove.json) -- list of faulty acquisitions to remove

Usefull to create the lists and testing, but must be moved into plugins for production

#### 1) Loading lists

In [ ]:
# This will load all defined curation lists
# Must be ececuted every time when lists are changed
from bidsme.plugins.tools.General import LoadCurationList, CleanupPrepared, CheckPrepared
white_list = LoadCurationList(curation_lists, "white_list")
black_list = LoadCurationList(curation_lists, "black_list", modalities=False)
remove_list = LoadCurationList(curation_lists, "to_remove")

#### 2) Cleaning-up/removing faulty recordings

In [ ]:
CleanupPrepared(prepared, remove_list)

#### 3) Checking if cleaned dataset is conforms to expectations

> If your dataset contains several standard sequences (for ex. if there was a protocol change in the middle of acquisition),
> you can specify several default acquisistion lists with defaults parameter: `defaults=["default1", "default2"]`

> If you want to check a specific subkect/session (for ex.to remove output), you can specify it using `sub` and `ses`parameters

> The strickness of checks can be regulated using `strict`, `complete` and `order` switches, for details, see the docstring 

In [ ]:
CheckPrepared(prepared, white_list)

# Map preparation

> To run on test dataset when bidsmap must be created/adjusted

 - [Bidsification plugin file](../plugin/bidsify_plugin.py)
 - [Bidsmap file](../map/bidsmap.yaml)
 - [Unknown recordings](../map/unknown.yaml)

In [ ]:
# Online help for creation of bidsmal
?bidsme.mapper

In [ ]:
# To manually set list of subjects to run, put here one or two subjects with new protocol
participants = ["sub-AVC012"]

In [ ]:
bidsme.mapper(prepared, bidsified,
              plugin_file=bidsify_plugin,
              bidsmapfile=bidsmap,
              sub_list=participants
              )

# Bidsification

> To run on test dataset when reconfiguration of bidsification is needed

 - [Bidsification plugin file](../plugin/bidsify_plugin.py)
 - [Bidsmap file](../maps/bidsmap.yaml)

In [ ]:
# Online help for bidsification
?bidsme.bidsify

In [ ]:
# Cleanup bidsified directory if needed
for f in os.listdir(bidsified):
    f = os.path.join(bidsified, f)
    if os.path.isdir(f):
        shutil.rmtree(f)
    else:
        os.remove(f)

In [ ]:
bidsme.bidsify(prepared, bidsified,
               plugin_file=bidsify_plugin,
               bidsmapfile=bidsmap,
               sub_skip_tsv=True,
               dry_run=False)